In [ ]:
!python3.11 -m pip install --upgrade pip

In [8]:
!pip3.11 install llama-index llama-cpp-python fpdf

In [9]:

from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)

from fpdf import FPDF

ImportError: cannot import name 'LlamaCPP' from 'llama_index.llms' (unknown location)

In [ ]:
model_url = 'llama-2-7b-chat.Q3_K_M.gguf'

In [ ]:
# LOAD MODEL 
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_path=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    # model_url=None,
    temperature=0.1,
    # max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    # context_window=4096,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    # model_kwargs={"n_gpu_layers": 1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

# llm = LlamaCPP(
#         model_url="https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q3_K_M.gguf",
#         model_path=None,
#         temperature=0,
#         max_new_tokens=512,
#         context_window=10000,
#         generate_kwargs={},
#         verbose=True,
#     )


In [ ]:
prompt ='''
Rewrite the user's accomplishment for a resume in the following format with full sentences. 

I accomplished X by the measure Y that resulted in Z


accomplishment: {}
'''


In [ ]:
basic_experience = '''
    As a Junior Developer at Norconsult, I've played a key role in enhancing our business intelligence app, implementing diverse visualizations using CraftJS, ChartJS, Leaflet, and React libraries. I improved data accessibility by designing a system to access various sources and storing data in PostgreSQL. Strengthening data analysis capabilities, I utilized CubeJS for querying, measures, and dimensions. Additionally, I developed a robust REST API using Node.js, Express, middleware, and Winston to enhance application functionality and reliability. I also contributed to AI and machine learning advancements through research and prototyping.

    My recent project, the Heads-Up Display Smart Helmet, received the "Best Safety Analysis" award, showcasing my innovation. I developed and tested it using Raspberry Pi 4 and Pi Camera for real-time video stream processing. Implementing object detection using TensorFlow lite models, I optimized performance with a transition from Python to C++. Demonstrating expertise in embedded systems design, I addressed task management through multiprocessing and multithreading.
'''
accomplishment=input("what did u achieve")

In [ ]:
final_question = prompt.join((accomplishment))

In [ ]:
response_iter = llm.stream_complete(final_question)
str_res = ""
for response in response_iter:
    str_res += response.delta
    print(response.delta, end="", flush=True)

In [ ]:
import textwrap

def text_to_pdf(text, filename):
    a4_width_mm = 210
    pt_to_mm = 0.35
    fontsize_pt = 10
    fontsize_mm = fontsize_pt * pt_to_mm
    margin_bottom_mm = 10
    character_width_mm = 7 * pt_to_mm
    width_text = a4_width_mm / character_width_mm

    pdf = FPDF(orientation='P', unit='mm', format='A4')
    pdf.set_auto_page_break(True, margin=margin_bottom_mm)
    pdf.add_page()
    pdf.set_font(family='Arial', size=fontsize_pt)
    splitted = text.split('\n')

    for line in splitted:
        lines = textwrap.wrap(line, width_text)

        if len(lines) == 0:
            pdf.ln()

        for wrap in lines:
            pdf.cell(0, fontsize_mm, wrap, ln=1)

    pdf.output(filename, 'F')

In [ ]:
print(str_res)
# save FPDF() class into a 
# variable pdf
pdf = FPDF()

text_to_pdf(str_res, "resume.pdf")